## Import Libraries

In [1]:
import numpy as np
import torch
import torch.nn.functional as f
import sounddevice as sd

from synth.ddsp_components import HarmonicSynth

## Init parameters for synthesis

In [2]:
# Init parameters
sr = 16000
duration = 2
n_harmonics = 5
n_samples = int(duration * sr)

# Create fundamental frequency envelope [n_batch, n_samples]
f0 = torch.ones([1, n_samples]) * 440 + torch.linspace(440, 0, n_samples)

# Create global amplitude envelope [n_batch, n_samples]
amp = torch.linspace(1, 0, n_samples)
amp = amp[np.newaxis, :]

# Create harmonic distribution [n_batch, n_samples, n_harmonics]
harm_distrib = torch.ones([1, n_samples, n_harmonics])
harm_distrib = f.normalize(harm_distrib, p=1, dim=2) # normalize distribution

## Synthesize sound from parameters

In [3]:
# Synthesize sound from inputs
harmonic_synth = HarmonicSynth(sr)
audio = harmonic_synth.get_signal(f0, amp, harm_distrib)

# Open the data stream
stream = sd.OutputStream(samplerate=sr, channels=1, dtype='float32')

# Start and write data on stream
stream.start()
stream.write(audio[0])

# Stop and close stream
stream.stop()
stream.close()